In [2]:
# Imports
import time
import torch
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.transforms as transforms  # Transformations and augmentations
from torch import optim  # For optimizers
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader #Dataloader module
from torch.utils.data import Dataset # Dataset module
import torchvision.transforms as transforms
from tqdm import tqdm #nice progress bar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Libraries for dataloader
from os import listdir
from os.path import isfile, join


#CUDA configuration
torch.cuda.empty_cache()
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'using: {device}')
torch.backends.cudnn.benchmark = True #let cudnn chose the most efficient way of calculating Convolutions
if device == 'cuda':
        #Checking whether GPU RAM is empty
        print('allocated CUDA memory: ',torch.cuda.memory_allocated())
        print('cached CUDA memory: ',torch.cuda.memory_cached())


using: cpu


In [3]:
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d_%m_%Y")
# path to save pytorch models during training
PATH = f'/notebooks/logs/model_{dt_string}.pt'

In [8]:
df_train=pd.read_csv('spaceship-titanic/train.csv')
df_xtest=pd.read_csv('spaceship-titanic/test.csv')
df_ytest=pd.read_csv('spaceship-titanic/sample_submission.csv')

xtrain=df_train.iloc[:,0:13].values  
ytrain=df_train.iloc[:,13].values
xtest=df_xtest.iloc[:,0:13].values
ytest=df_ytest.iloc[:,1].values

for i in [5,7,8,9,10,11]:  #normalizing numeric values by their biggest value
    xtest[:,i] = xtest[:,i]/max(xtrain[:,i])
    xtrain[:,i] = xtrain[:,i]/max(xtrain[:,i])

df_train[0:3]


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False


In [5]:
# Template of function, modify for proper use
# for calcualting mean and std od dataset
def get_mean_and_std(dataset):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in tqdm(dataset):
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[1,2])
        channels_squared_sum += torch.mean(data**2, dim=[1,2])
        num_batches += 1

    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [6]:
# Network prototype
class model_block(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(model_block, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)
    def forward(self, x):
        y = self.linear1(x)
        y = F.relu(y)
        y = self.linear2(x)
        y = F.relu(y)
        return y

class Net(torch.nn.Module):
    def __init__(self, D_in1, H1, D_out1, D_in2, H2, D_out2):
        super(Net, self).__init__()
        self.block1 = model_block(D_in1, H1, D_out1)
        self.block2 = model_block(D_in2, H2, D_out2)

    def forward(self, x):
        y = self.block1(x)
        y = self.block2(y)
        return y

input_num = 4
output_num = 1

SyntaxError: invalid syntax (640636866.py, line 8)